Ridge regression with splines and interactions on MDA_dataframeArthur.csv

Time Series cross-validation: 24h delay for weather, 1 month gap between train and test sets.
MAE en RMSE can be compared to other models.

In [1]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import TimeSeriesSplit
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import cross_validate
import sklearn.model_selection
from sklearn import*
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import RidgeCV
from sklearn.preprocessing import SplineTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import FeatureUnion

Dataframe from MDA_merge_data.ipynb is loaded, datetime column is dropped in order to run model.

In [2]:
df = pd.read_csv("C:\\Users\Arthur\hello\\final_df.csv", delimiter=";")
df['255439'] = df['object_id']==255439
df['255440'] = df['object_id']==255440
df['255441'] = df['object_id']==255441
df['255442'] = df['object_id']==255442
df['255443'] = df['object_id']==255443
df['255444'] = df['object_id']==255444
df['255445'] = df['object_id']==255445
df['280324'] = df['object_id']==280324
df['303910'] = df['object_id']==303910
df = df.sort_values(by=['result_timestamp','object_id'])


In [3]:
df['key'] = df['object_id'].astype('str') + df['result_timestamp']
df['result_timestamp'] = pd.to_datetime(df['result_timestamp'],format='%Y-%m-%d %H:%M:%S')


In [4]:
df_noisedelay = df[['object_id','result_timestamp','laeq']]
df_noisedelay['result_timestamp'] = df_noisedelay['result_timestamp']+pd.DateOffset(days=1)
df_noisedelay['laeq_delayed'] = df_noisedelay['laeq']
df_noisedelay['key'] = df_noisedelay['object_id'].astype('str') + df_noisedelay['result_timestamp'].astype('str')
df_noisedelay = df_noisedelay.drop(['laeq','object_id','result_timestamp'], axis=1)

df_weatherdelay = df[['object_id','result_timestamp','LC_HUMIDITY','LC_DWPTEMP','LC_n','LC_RAD','LC_RAININ',
                   'LC_DAILYRAIN','LC_WINDDIR','LC_WINDSPEED','LC_RAD60','LC_TEMP_QCL0']]
df_weatherdelay['result_timestamp'] = df_weatherdelay['result_timestamp']+pd.DateOffset(days=1)
df_weatherdelay['key'] = df_weatherdelay['object_id'].astype('str') + df_weatherdelay['result_timestamp'].astype('str')
df_weatherdelay = df_weatherdelay.drop(['object_id','result_timestamp'], axis=1)
df = df.drop(['LC_HUMIDITY','LC_DWPTEMP','LC_n','LC_RAD','LC_RAININ',
                'LC_DAILYRAIN','LC_WINDDIR','LC_WINDSPEED','LC_RAD60','LC_TEMP_QCL0'],axis=1)


C:\Users\Arthur\AppData\Local\Temp\ipykernel_20144\727566501.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_noisedelay['result_timestamp'] = df_noisedelay['result_timestamp']+pd.DateOffset(days=1)
C:\Users\Arthur\AppData\Local\Temp\ipykernel_20144\727566501.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_noisedelay['laeq_delayed'] = df_noisedelay['laeq']
C:\Users\Arthur\AppData\Local\Temp\ipykernel_20144\727566501.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

In [5]:
df = pd.merge(df,df_noisedelay, on='key',validate='1:1')
df= pd.merge(df,df_weatherdelay, on='key',validate='1:1')
df_nd = df.drop(['key','object_id','result_timestamp','date','time','day_time','day_hour','LC_TEMP_QCL1','LC_TEMP_QCL2','LC_TEMP_QCL3'], axis=1)

In [6]:
df_nd.isna().sum()


laeq             0
hour             0
weekday          0
month            0
weekend          0
holiday          0
exam_period      0
255439           0
255440           0
255441           0
255442           0
255443           0
255444           0
255445           0
280324           0
303910           0
laeq_delayed     0
LC_HUMIDITY     43
LC_DWPTEMP      43
LC_n            43
LC_RAD          43
LC_RAININ       43
LC_DAILYRAIN    43
LC_WINDDIR      43
LC_WINDSPEED    43
LC_RAD60         4
LC_TEMP_QCL0    43
dtype: int64

In [7]:
df_nd = df_nd[df_nd.isna()['LC_HUMIDITY']==False]
df_nd.isna().sum()

laeq            0
hour            0
weekday         0
month           0
weekend         0
holiday         0
exam_period     0
255439          0
255440          0
255441          0
255442          0
255443          0
255444          0
255445          0
280324          0
303910          0
laeq_delayed    0
LC_HUMIDITY     0
LC_DWPTEMP      0
LC_n            0
LC_RAD          0
LC_RAININ       0
LC_DAILYRAIN    0
LC_WINDDIR      0
LC_WINDSPEED    0
LC_RAD60        0
LC_TEMP_QCL0    0
dtype: int64

In [8]:
df_nd['weekday'] = df_nd['weekday'].replace(to_replace='Monday', value=0)
df_nd['weekday'] = df_nd['weekday'].replace(to_replace='Tuesday', value=1)
df_nd['weekday'] = df_nd['weekday'].replace(to_replace='Wednesday', value=2)
df_nd['weekday'] = df_nd['weekday'].replace(to_replace='Thursday', value=3)
df_nd['weekday'] = df_nd['weekday'].replace(to_replace='Friday', value=4)
df_nd['weekday'] = df_nd['weekday'].replace(to_replace='Saturday', value=5)
df_nd['weekday'] = df_nd['weekday'].replace(to_replace='Sunday', value=6)

The target dataset (y) and predictor dataset (X) are created. For y, noise is converted to relative noise (proportion of max value)

In [9]:
y = df_nd["laeq"]
X = df_nd.drop(["laeq"], axis="columns")

In [10]:
#create time-sensitive split for cross-validation
ts_cv = TimeSeriesSplit(
    n_splits=5,
    gap=38880,
    max_train_size=66000,
    test_size=30000,
)

In [11]:
#inspect splits
all_splits = list(ts_cv.split(X, y))
train_0, test_0 = all_splits[0]
X.iloc[test_0]
X.iloc[train_0]

,hour,weekday,month,weekend,holiday,exam_period,255439,255440,255441,255442,...,LC_HUMIDITY,LC_DWPTEMP,LC_n,LC_RAD,LC_RAININ,LC_DAILYRAIN,LC_WINDDIR,LC_WINDSPEED,LC_RAD60,LC_TEMP_QCL0
75280,13,5,4,True,True,False,False,False,False,False,...,56.0,8.70,32.0,420.0,0.0,0.0,110.0,0.37,496.0,17.65
75281,13,5,4,True,True,False,False,False,False,False,...,56.0,8.70,32.0,420.0,0.0,0.0,110.0,0.37,496.0,17.65
75282,13,5,4,True,True,False,False,False,False,False,...,56.0,8.70,32.0,420.0,0.0,0.0,110.0,0.37,496.0,17.65
75283,13,5,4,True,True,False,True,False,False,False,...,56.0,8.57,32.0,92.0,0.0,0.0,-44.0,0.05,492.0,17.57
75284,13,5,4,True,True,False,False,False,True,False,...,56.0,8.57,32.0,92.0,0.0,0.0,-44.0,0.05,492.0,17.57
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141275,1,1,6,False,False,True,False,False,False,False,...,82.0,11.85,31.0,0.0,0.0,0.0,111.0,0.05,0.0,14.76
141276,1,1,6,False,False,True,True,False,False,False,...,82.0,11.61,32.0,0.0,0.0,0.0,83.0,0.09,0.0,14.60
141277,1,1,6,False,False,True,False,True,False,False,...,82.0,11.61,32.0,0.0,0.0,0.0,83.0,0.09,0.0,14.60
141278,1,1,6,False,False,True,False,False,True,False,...,82.0,11.61,32.0,0.0,0.0,0.0,83.0,0.09,0.0,14.60


In [12]:
train_4, test_4 = all_splits[4]
X.iloc[test_4]
X.iloc[train_4]

,hour,weekday,month,weekend,holiday,exam_period,255439,255440,255441,255442,...,LC_HUMIDITY,LC_DWPTEMP,LC_n,LC_RAD,LC_RAININ,LC_DAILYRAIN,LC_WINDDIR,LC_WINDSPEED,LC_RAD60,LC_TEMP_QCL0
195280,14,5,8,True,True,False,False,False,False,False,...,21.0,7.93,32.0,550.0,0.0,0.0,94.0,0.22,578.0,33.04
195281,15,5,8,True,True,False,True,False,False,False,...,21.0,7.92,31.0,545.0,0.0,0.0,87.0,0.63,572.0,33.16
195282,15,5,8,True,True,False,False,True,False,False,...,21.0,7.92,31.0,545.0,0.0,0.0,87.0,0.63,572.0,33.16
195283,15,5,8,True,True,False,False,False,True,False,...,21.0,7.92,31.0,545.0,0.0,0.0,87.0,0.63,572.0,33.16
195284,15,5,8,True,True,False,False,False,False,True,...,21.0,7.92,31.0,545.0,0.0,0.0,87.0,0.63,572.0,33.16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261314,13,0,10,False,False,False,False,False,False,False,...,70.0,14.75,37.0,114.0,0.0,0.0,-176.0,0.10,115.0,20.33
261315,13,0,10,False,False,False,False,False,False,False,...,70.0,14.75,37.0,114.0,0.0,0.0,-176.0,0.10,115.0,20.33
261316,13,0,10,False,False,False,False,False,False,False,...,70.0,14.75,37.0,114.0,0.0,0.0,-176.0,0.10,115.0,20.33
261317,13,0,10,False,False,False,False,False,False,False,...,70.0,14.75,37.0,114.0,0.0,0.0,-176.0,0.10,115.0,20.33


In [13]:
def evaluate(model, X, y, cv):
    cv_results = cross_validate(
        model,
        X,
        y,
        cv=cv,
        scoring=["neg_mean_absolute_error", "neg_root_mean_squared_error"],
    )
    mae = -cv_results["test_neg_mean_absolute_error"]
    rmse = -cv_results["test_neg_root_mean_squared_error"]
    print(
        f"Mean Absolute Error:     {mae.mean():.3f} +/- {mae.std():.3f}\n"
        f"Root Mean Squared Error: {rmse.mean():.3f} +/- {rmse.std():.3f}"
    )


In [14]:
X

,hour,weekday,month,weekend,holiday,exam_period,255439,255440,255441,255442,...,LC_HUMIDITY,LC_DWPTEMP,LC_n,LC_RAD,LC_RAININ,LC_DAILYRAIN,LC_WINDDIR,LC_WINDSPEED,LC_RAD60,LC_TEMP_QCL0
4,0,6,1,True,True,False,False,False,True,False,...,91.0,11.88,38.0,0.0,0.0,0.0,-48.0,0.04,0.0,13.28
5,0,6,1,True,True,False,False,False,False,True,...,91.0,11.88,38.0,0.0,0.0,0.0,-48.0,0.04,0.0,13.28
6,0,6,1,True,True,False,False,False,False,False,...,91.0,11.88,38.0,0.0,0.0,0.0,-48.0,0.04,0.0,13.28
7,0,6,1,True,True,False,False,False,False,False,...,91.0,11.88,38.0,0.0,0.0,0.0,-48.0,0.04,0.0,13.28
8,0,6,1,True,True,False,False,False,True,False,...,90.0,11.72,37.0,0.0,0.0,0.0,45.0,0.02,0.0,13.28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
330194,23,5,12,True,True,False,False,False,False,True,...,95.0,12.78,38.0,0.0,0.0,0.0,-167.0,0.02,0.0,13.61
330195,23,5,12,True,True,False,False,False,False,False,...,95.0,12.78,38.0,0.0,0.0,0.0,-167.0,0.02,0.0,13.61
330196,23,5,12,True,True,False,False,False,False,False,...,95.0,12.78,38.0,0.0,0.0,0.0,-167.0,0.02,0.0,13.61
330197,23,5,12,True,True,False,False,False,False,False,...,95.0,12.78,38.0,0.0,0.0,0.0,-167.0,0.02,0.0,13.61


In [15]:
min_max_scaler=MinMaxScaler()
numerical_columns=['hour','weekday','month','weekend','holiday','exam_period','LC_HUMIDITY','LC_DWPTEMP','LC_n','LC_RAD','LC_RAININ',
                   'LC_DAILYRAIN','LC_WINDDIR','LC_WINDSPEED','LC_RAD60','LC_TEMP_QCL0','laeq_delayed']
alphas = np.logspace(-6, 6, 25)
naive_linear_pipeline = make_pipeline(
    ColumnTransformer(
        transformers=[
            ("minmax", min_max_scaler, numerical_columns),
        ],remainder="passthrough",
    ),
    RidgeCV(alphas=alphas),
)
evaluate(naive_linear_pipeline, X, y, cv=ts_cv)

Mean Absolute Error:     3.307 +/- 0.404
Root Mean Squared Error: 4.367 +/- 0.438


In [16]:
def periodic_spline_transformer(period, n_splines=None, degree=3):
    if n_splines is None:
        n_splines = period
    n_knots = n_splines + 1  # periodic and include_bias is True
    return SplineTransformer(
        degree=degree,
        n_knots=n_knots,
        knots=np.linspace(0, period, n_knots).reshape(n_knots, 1),
        extrapolation="periodic",
        include_bias=True,
    )
cyclic_spline_transformer = ColumnTransformer(
    transformers=[
        ("cyclic_month", periodic_spline_transformer(12, n_splines=6), ["month"]),
        ("cyclic_weekday", periodic_spline_transformer(7, n_splines=3), ["weekday"]),
        ("cyclic_hour", periodic_spline_transformer(24, n_splines=12), ["hour"]),
    ],
    remainder=MinMaxScaler(),
)
cyclic_spline_linear_pipeline = make_pipeline(
    cyclic_spline_transformer,
    RidgeCV(alphas=alphas),
)
evaluate(cyclic_spline_linear_pipeline, X, y, cv=ts_cv)

Mean Absolute Error:     3.476 +/- 0.602
Root Mean Squared Error: 4.466 +/- 0.522


In [17]:
hour_weekend_interaction = make_pipeline(
    ColumnTransformer(
        [
            ("cyclic_hour", periodic_spline_transformer(24, n_splines=8), ["hour"]),
            ("weekend", FunctionTransformer(lambda x: x == 1), ["weekend"]),
        ]
    ),
    PolynomialFeatures(degree=2, interaction_only=True, include_bias=False),
)

In [18]:
#adding combining the spline pipeline and the interactions pipeline
cyclic_spline_interactions_pipeline = make_pipeline(
    FeatureUnion(
        [
            ("marginal", cyclic_spline_transformer),
            ("interactions1", hour_weekend_interaction),            
        ]
    ),
    RidgeCV(alphas=alphas),
)
evaluate(cyclic_spline_interactions_pipeline, X, y, cv=ts_cv)

Mean Absolute Error:     3.340 +/- 0.430
Root Mean Squared Error: 4.324 +/- 0.362


In [19]:
from sklearn.kernel_approximation import Nystroem

cyclic_spline_poly_pipeline = make_pipeline(
    cyclic_spline_transformer,
    Nystroem(kernel="poly", degree=2, n_components=70, random_state=0),
    RidgeCV(alphas=alphas),
)
evaluate(cyclic_spline_poly_pipeline, X, y, cv=ts_cv)

Mean Absolute Error:     3.740 +/- 0.684
Root Mean Squared Error: 4.692 +/- 0.618
